In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 


import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import lightgbm as lgbm
from xgboost import plot_importance
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test_supplement.csv', 'train.csv', 'test.csv', 'sample_submission.csv', 'train_sample.csv']


In [2]:
is_valid = False

path = '../input/'

train_columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_columns  = ['ip', 'app', 'device', 'os', 'channel', 'click_time']
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

# Read the last lines because they are more impacting in training than the starting lines
train = pd.read_csv(path+"train.csv", skiprows=range(1,131886954), nrows=53016937, usecols=train_columns, dtype=dtypes)
test = pd.read_csv(path+"test.csv", usecols=test_columns, dtype=dtypes)


In [ ]:
test.head()

In [3]:
Y=train.is_attributed
train.drop('is_attributed',axis=1,inplace=True)

In [4]:
def timeFeatures(df):
    # Make some new features with click_time column
    df['datetime'] = pd.to_datetime(df['click_time'])
    df['dow']      = df['datetime'].dt.dayofweek
    df["hour"]      = df["datetime"].dt.hour
    #df["dteom"]    = df["datetime"].dt.daysinmonth - df["datetime"].dt.day
    df.drop(['click_time', 'datetime'], axis=1, inplace=True)
    return df


In [5]:
nrow_train = train.shape[0]
merge = pd.concat([train, test])


In [6]:
%%time

#del train, test
gc.collect()

# Count the number of clicks by ip
ip_count = merge.groupby(['ip'])['channel'].count().reset_index()
ip_count.columns = ['ip', 'clicks_by_ip']
merge = pd.merge(merge, ip_count, on='ip', how='left', sort=False)
merge['clicks_by_ip'] = merge['clicks_by_ip'].astype('uint16')
merge.drop('ip', axis=1, inplace=True)
del ip_count
print('ip done')
# Count the number of os by ip
ip_count = merge.groupby(['channel'])['os'].count().reset_index()
ip_count.columns = ['channel', 'clicks_by_channel']
merge = pd.merge(merge, ip_count, on='channel', how='left', sort=False)
merge['clicks_by_channel'] = merge['clicks_by_channel'].astype('uint16')
#merge.drop('channel', axis=1, inplace=True)
del ip_count
print('channel done')

ip_count = merge.groupby(['os'])['channel'].count().reset_index()
ip_count.columns = ['os', 'clicks_by_os']
merge = pd.merge(merge, ip_count, on='os', how='left', sort=False)
merge['clicks_by_os'] = merge['clicks_by_os'].astype('uint16')
#merge.drop('os', axis=1, inplace=True)
del ip_count
print('os done')
ip_count = merge.groupby(['app'])['channel'].count().reset_index()
ip_count.columns = ['app', 'clicks_by_app']
merge = pd.merge(merge, ip_count, on='app', how='left', sort=False)
merge['clicks_by_app'] = merge['clicks_by_app'].astype('uint16')
#merge.drop('app', axis=1, inplace=True)
del ip_count

print('app done')

train = merge[:nrow_train]
test = merge[nrow_train:]

#del ip_count,merge
#gc.collect()

ip done
channel done
os done
app done
CPU times: user 2min 9s, sys: 2min 29s, total: 4min 39s
Wall time: 2min 30s


In [7]:
del merge
gc.collect()

890

In [8]:
test=timeFeatures(test)

In [9]:
train=timeFeatures(train)


In [10]:
train.head()

,app,device,os,channel,clicks_by_ip,clicks_by_channel,clicks_by_os,clicks_by_app,dow,hour
0,11,1,13,487,973,22788,40127,48238,3,0
1,2,1,13,469,1115,20075,40127,59188,3,0
2,26,1,19,477,1079,35638,29757,2827,3,0
3,18,1,8,121,2211,65389,193,11842,3,0
4,12,1,19,265,11980,17596,29757,4903,3,0


In [11]:
test.head()

,app,device,os,channel,clicks_by_ip,clicks_by_channel,clicks_by_os,clicks_by_app,dow,hour
53016937,9,1,3,107,469,40721,26647,42174,4,4
53016938,9,1,3,466,7918,21138,26647,42174,4,4
53016939,21,1,19,128,5719,22225,29757,32857,4,4
53016940,15,1,13,111,4513,31367,40127,54851,4,4
53016941,12,1,13,328,583,30346,40127,4903,4,4


In [15]:
train.tail()

,app,device,os,channel,clicks_by_ip,clicks_by_channel,clicks_by_os,clicks_by_app,dow,hour
53016932,12,1,10,340,1270,45951,29344,4903,3,16
53016933,3,1,19,211,2782,245,29757,6979,3,16
53016934,1,1,13,274,493,5670,40127,30300,3,16
53016935,12,1,37,259,871,65217,37281,4903,3,16
53016936,11,1,19,137,8275,23245,29757,48238,3,16


In [29]:
x_valid=train[:52486768]
y_valid=Y[:52486768]
x_train=train[52486768:]
y_train=Y[52486768:]

In [31]:
x_train.shape,x_valid.shape,y_train.shape,y_valid.shape

((530169, 10), (52486768, 10), (530169,), (52486768,))

In [33]:
    
dtrain = lgbm.Dataset(x_train, label=y_train)
dvalid = lgbm.Dataset(x_valid, label=y_valid    )

In [34]:
gc.collect()

200

In [ ]:

params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.1,
    'num_leaves':7,  
    'max_depth': 4,
    'scale_pos_weight': 99.7
    }


evals_results = {}

print("Training the model...")

lgb_model = lgbm.train(params, dtrain,valid_sets=[dtrain, dvalid], 
                 valid_names=['train','valid'], 
                 evals_result=evals_results, 
                 num_boost_round=500)

Training the model...


In [ ]:
print('Feature names:', lgb_model.feature_name())
print('Feature importances:', list(lgb_model.feature_importance()))


In [ ]:
submit = pd.read_csv(path_test, dtype='int', usecols=['click_id'])
submit['is_attributed'] = lgb_model.predict(test[predictors], num_iteration=lgb_model.best_iteration)
submit.to_csv('submission.csv', index=False)


In [ ]:

del train
del val
gc.collect()
